In [11]:
import matplotlib
import numpy as np
import torch
import torch.nn as nn

import argparse
import os
import pickle
import random
import shutil
import time

from PIL import Image

import landsat_prep as lp
import geograph as gg

/home/hbaier/anaconda3/envs/caoe/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [24]:
gl = gg.ImageGraphLoader("./data/", "MEX", "./migration_data.json", 1)#.data

In [14]:
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
import torch


class resnet18_mod(torch.nn.Module):

    def __init__(self, resnet):
        super().__init__()
        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        self.avgpool = resnet.avgpool
        self.fc = resnet.fc

    def forward(self, x):

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.flatten(start_dim=1)
        # out = torch.cat((out, ids, coords), dim = 1)
        # out = self.fc(out)
        return out

In [15]:
r18 = models.resnet18(pretrained = True)
model = resnet18_mod(r18)
input = model(final_im)#.detach().numpy()
input.shape

NameError: name 'models' is not defined

In [16]:
# GCN basic operation
class GraphConv(nn.Module):
    def __init__(self, input_dim, output_dim, bias=True):
        super(GraphConv, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.weight = nn.Parameter(torch.rand(input_dim, output_dim))#.cuda())
        if bias:
            self.bias = nn.Parameter(torch.rand(output_dim))#.cuda())
        else:
            self.bias = None

    def forward(self, x, adj):
        y = torch.matmul(adj, x)
        y = torch.matmul(y, self.weight)
        if self.bias is not None:
            y = y + self.bias
        return y

# Generate Assignment Matrix
class GenAssign(nn.Module):
    def __init__(self, input_dim, num_clusters, bias=True):
        super(GenAssign, self).__init__()
        self.input_dim = input_dim
        self.num_clusters = num_clusters
        self.weight = nn.Parameter(torch.rand(input_dim, num_clusters))#.cuda())
        if bias:
            self.bias = nn.Parameter(torch.rand(num_clusters))#.cuda())
        else:
            self.bias = None
        # self.linear = torch.nn.Linear(input_dim, num_clusters)
        self.sm = torch.nn.Softmax(dim = 1)

    def forward(self, x, adj):
        y = torch.matmul(adj, x)
        y = torch.matmul(y, self.weight)
        if self.bias is not None:
            y = y + self.bias
        # y = self.linear(y)
        y = self.sm(y)
        _, i = torch.max(y, 1)
        return i


# Pool based on clusters
class PoolClusters(nn.Module):
    def __init__(self):
        super(PoolClusters, self).__init__()

    def forward(self, x, sl):
        cluster_ids = torch.unique(sl)
        for cluster in cluster_ids:
            indices = torch.nonzero(sl == cluster, as_tuple = True)[0]
            cluster_mean = torch.mean(torch.index_select(x, 0, indices), dim = 0).unsqueeze(0)
            try:
                out = torch.cat((out, cluster_mean))
            except Exception as e:
                out = cluster_mean
        return out


def make_edge_list(neighbors_dict):
    edge_list = []
    for k,v in neighbors_dict.items():
        [edge_list.append([k, cur_v]) for cur_v in v]
    return edge_list

def make_adj_matrix(edge_list, dim):
    adj_matrix = np.zeros((dim, dim))
    for edge in edge_list:
        adj_matrix[edge[0]][edge[1]] = 1
    for i in range(dim):
        adj_matrix[i][i] = 1

    return adj_matrix


def fix_sl(sl):
    cluster_ids = torch.unique(sl)
    ref = dict(zip([i.item() for i in cluster_ids], [i for i in range(len(cluster_ids))]))
    for i in cluster_ids:
        sl[sl == i.item()] = ref[i.item()]
    return sl


# GCN basic operation
class GenAdj(nn.Module):
    def __init__(self):
        super(GenAdj, self).__init__()

    def forward(self, adj, sl, neighbors_dict):

        sl = fix_sl(sl)

        adj_map, new_neighbors_dict = {}, {}
        cluster_ids = torch.unique(sl)

        # Create a dictionary where each key is a node and the value is the cluster it belongs to
        for cluster in cluster_ids:
            indices = torch.nonzero(sl == cluster, as_tuple = True)[0]
            for i in indices:
                if i not in adj_map.keys():
                    adj_map[i.item()] = cluster.item()

        # Create a new neighbors dictionary for each of the clusters
        for cluster in cluster_ids:
            indices = torch.nonzero(sl == cluster, as_tuple = True)[0]

            for i in indices:
                temp = []
                [temp.append(adj_map[i]) for i in neighbors_dict[i.item()] if i not in temp]
                temp = list(set(temp)) # TO-DO: HERE CREATE SOME SORT OF SECONDARY DICTIONARY OR SOMETHING THAT COUNTS HOW MANY EDGES TO CREATE WEIGHTED ADJ MATRIX

                new_neighbors_dict[cluster.item()] = temp

        edge_list = make_edge_list(new_neighbors_dict)
        adj_matrix = make_adj_matrix(edge_list, len(cluster_ids))

        return torch.tensor(adj_matrix, dtype = torch.float32), new_neighbors_dict


class diffPool(nn.Module):
    def __init__(self, input_dim, output_dim, num_clusters, bias=True):
        super(diffPool, self).__init__()

        self.gc = GraphConv(input_dim = input_dim, output_dim = output_dim)
        self.bn = torch.nn.BatchNorm1d(output_dim)
        self.gen_assign = GenAssign(input_dim = output_dim, num_clusters = num_clusters)
        self.pool = PoolClusters()
        self.gen_adj = GenAdj()

    def forward(self, x, adj, neighbors_dict):

        # Generate node embeddings & normalize (no pooling yet)
        x = self.gc(x, torch.tensor(adj, dtype = torch.float32))
        x = self.bn(x)
        # Create the assignment matrix (i.e. assign each node to a cluster)
        sl = self.gen_assign(x, adj)
        # Create the pooled feature matrix
        x = self.pool(x, sl)
        # Generate the new adjacency matrix
        adj, neighbors_dict = self.gen_adj(adj, sl, neighbors_dict)
        # print(neighbors_dict)

        return x, adj, neighbors_dict

In [27]:
class diffPool(nn.Module):
    def __init__(self, input_dim, output_dim, num_clusters, bias=True):
        super(diffPool, self).__init__()

        self.gc = GraphConv(input_dim = input_dim, output_dim = output_dim)
        self.bn = torch.nn.BatchNorm1d(output_dim)
        self.gen_assign = GenAssign(input_dim = output_dim, num_clusters = num_clusters)
        self.pool = PoolClusters()
        self.gen_adj = GenAdj()

    def forward(self, x, adj, neighbors_dict):

        # Generate node embeddings & normalize (no pooling yet)
        x = self.gc(x, torch.tensor(adj, dtype = torch.float32))
        if x.shape[0] > 1:
            x = self.bn(x)

        # print(x.shape)
        # Create the assignment matrix (i.e. assign each node to a cluster)
        sl = self.gen_assign(x, adj)
        # Create the pooled feature matrix
        x = self.pool(x, sl)
        # Generate the new adjacency matrix
        adj, neighbors_dict = self.gen_adj(adj, sl, neighbors_dict)
        # print(neighbors_dict)

        return x, adj, neighbors_dict

In [37]:
class spatialPool(nn.Module):
    def __init__(self, input_dim, output_dim, num_clusters, resnet, bias=True):
        super(spatialPool, self).__init__()

        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        self.avgpool = resnet.avgpool

        self.dp1 = diffPool(input_dim = input_dim, output_dim = output_dim, num_clusters = 4)
        self.dp2 = diffPool(input_dim = output_dim, output_dim = 128, num_clusters = 2)
        self.dp3 = diffPool(input_dim = 128, output_dim = 256, num_clusters = 1)
        self.linear = torch.nn.Linear(256, 1)

    def forward(self, x, adj, neighbors_dict):

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.flatten(start_dim=1)

        # print(x.shape)

        x, adj, neighbors_dict = self.dp1(x, adj, neighbors_dict)
        x, adj, neighbors_dict = self.dp2(x, adj, neighbors_dict)
        x, adj, neighbors_dict = self.dp3(x, adj, neighbors_dict)
        x = self.linear(x)

        return x

In [38]:
from torchvision import models

In [43]:
r18 = models.resnet18(pretrained = True)
model = spatialPool(input_dim = 512, output_dim = 256, num_clusters = 4, resnet = r18)
criterion = torch.nn.L1Loss(reduction = "mean")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

final_im = gl.data[0].x
adj = gl.data[0].adj_matrix
neighbors_dict = gl.data[0].neighbors
y = gl.data[0].y
batch = gl.data[0].batch

for epoch in range(0, 50):  

    running_loss, c = 0, 1

    for i in range(0, 4):

        final_im = gl.data[i].x
        adj = gl.data[i].adj_matrix
        neighbors_dict = gl.data[i].neighbors
        y = gl.data[i].y
        batch = gl.data[i].batch

        

        pred = model(final_im, torch.tensor(adj, dtype = torch.float32), neighbors_dict)
        # print(pred)
        loss = criterion(pred, torch.tensor([y]))

        print("PRED: ", round(pred.item(), 2),  "  |  LOSS: ", round(loss.item(), 2))

        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        c += 1

    print(running_loss / c, "\n")

PRED:  -0.04   |  LOSS:  42055.04
PRED:  10.67   |  LOSS:  3406.33
PRED:  0.1   |  LOSS:  11991.9
PRED:  30.11   |  LOSS:  3690.89
12228.830615234376 

PRED:  -0.01   |  LOSS:  42055.01
PRED:  58.56   |  LOSS:  3358.44
PRED:  0.66   |  LOSS:  11991.34
PRED:  96.48   |  LOSS:  3624.52
12205.862841796876 

PRED:  0.01   |  LOSS:  42054.99
PRED:  144.51   |  LOSS:  3272.49
PRED:  1.43   |  LOSS:  11990.57
PRED:  1.67   |  LOSS:  3719.33
12207.474951171875 

PRED:  0.03   |  LOSS:  42054.97
PRED:  258.51   |  LOSS:  3158.49
PRED:  2.45   |  LOSS:  11989.55
PRED:  328.45   |  LOSS:  3392.55
12119.1119140625 

PRED:  0.05   |  LOSS:  42054.95
PRED:  413.19   |  LOSS:  3003.81
PRED:  3.76   |  LOSS:  11988.24
PRED:  513.19   |  LOSS:  3207.81
12050.96201171875 

PRED:  0.07   |  LOSS:  42054.93
PRED:  629.06   |  LOSS:  2787.94
PRED:  5.39   |  LOSS:  11986.61
PRED:  5.86   |  LOSS:  3715.14
12108.9216796875 

PRED:  6.37   |  LOSS:  42048.63
PRED:  881.89   |  LOSS:  2535.11
PRED:  7.49   | 

KeyboardInterrupt: 

In [31]:
pred

tensor([[5055.0986]], grad_fn=<AddmmBackward>)